In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import clear_output
!pip install imutils
clear_output()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, keras
import shutil
import cv2

import imutils
from PIL import Image
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import glob
import tensorflow as tf

from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle

In [ ]:
from numpy.random import seed
seed(88)
from tensorflow.random import set_seed
set_seed(88)

In [ ]:
def crop_imgs(set_name, add_pixels_value=0):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    set_new = []
    for img in set_name:
        img = cv2.imread(img) #original syntax
        #image = load_img(img,
                         #color_mode = 'grayscale')
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) #original syntax
        #gray = np.array(image)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = add_pixels_value
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        set_new.append(new_img)

    return np.array(set_new)

def crop_imgs_array(set_name, add_pixels_value=0):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    set_new = []
    for img in set_name:
        gray = cv2.GaussianBlur(img, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = add_pixels_value
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        set_new.append(new_img)

    return np.array(set_new)

def gray_scale_img(path) :
    for sub_dir in tqdm(os.listdir(path)) :
        for file in os.listdir(path + '/' + sub_dir) :
            src = load_img(path + '/' + sub_dir + '/' + file,
                           color_mode = 'grayscale')
            #src = cv2.imread(path + '/' + sub_dir + '/' + file)
            output = np.array(src)
            cv2.imwrite(path + '/' + sub_dir + '/' + file, output)

In [ ]:
#os.listdir('./VAL')
def resize_img_path(path, dsize) :
    for sub_dir in tqdm(os.listdir(path)) :
        for file in os.listdir(path + '/' + sub_dir) :
            src = cv2.imread(path + '/' + sub_dir + '/' + file)
            output = cv2.resize(src, dsize)
            cv2.imwrite(path + '/' + sub_dir + '/' + file,output)

In [ ]:
#os.listdir('./VAL')
def resize_img(path, dsize) :
    result = []
    #for sub_dir in tqdm(os.listdir(path)) :
    for file in os.listdir(path) :
        src = cv2.imread(path + '/' + file)
        output = cv2.resize(src, dsize)
        #cv2.imwrite(path + '/' + sub_dir + '/' + file,output)
        result.append(output)
    return np.array(result)

In [ ]:
def resize_img_array(data, dsize) :
    result = []
    #for sub_dir in tqdm(os.listdir(path)) :
    for img in data :
        output = cv2.resize(img, dsize)
        result.append(output)
    return np.array(result)

In [ ]:
def prob_converter(data) :
    dim = data[0].shape[0]
    result = []
    prob = []
    for i in range(dim) :
        res = []
        res.append(np.mean((data[0][i][0], data[1][i][0], data[2][i][0])))
        res.append(np.mean((data[0][i][1], data[1][i][1], data[2][i][1])))
        res.append(np.mean((data[0][i][2], data[1][i][2], data[2][i][2])))
        res.append(np.mean((data[0][i][3], data[1][i][3], data[2][i][3])))
        prob.append(res)
    prob = np.array(prob)
    
    for i in prob :
        result.append(np.where(i == i.max())[0][0])
    return result

def prob_average(data) :
    dim = data[0].shape[0]
    result = []
    prob = []
    for i in range(dim) :
        res = []
        res.append(np.mean((data[0][i][0], data[1][i][0], data[2][i][0])))
        res.append(np.mean((data[0][i][1], data[1][i][1], data[2][i][1])))
        res.append(np.mean((data[0][i][2], data[1][i][2], data[2][i][2])))
        res.append(np.mean((data[0][i][3], data[1][i][3], data[2][i][3])))
        prob.append(res)
    prob = np.array(prob)
    
    return prob

In [ ]:
IMG_PATH = '../input/brain-mri-os'
#for Class in os.listdir(IMG_PATH) :
#    print(Class)
print('glioma_tumor : ' + str(len([file for file in os.listdir('../input/brain-mri-os/Training fix/glioma_tumor_fix')])))
print('meningioma_tumor : ' + str(len([file for file in os.listdir('../input/brain-mri-os/Training fix/meningioma_tumor_fix')])))
print('hipofisis tumor : ' + str(len([file for file in os.listdir('../input/brain-mri-os/Training fix/pituitary_tumor_fix')])))
print('no tumor : ' + str(len([file for file in os.listdir('../input/brain-mri-os/Training fix/no_tumor_fix')])))

In [ ]:
IMG_PATH = '../input/brain-mri-os/Training fix/'
img_file = []
label = []
size = []
path_img = []
for path in os.listdir(IMG_PATH) :    
    for item in os.listdir(IMG_PATH + str(path)) :
        img_file.append(item)
        label.append(str(path))
        size.append(Image.open(IMG_PATH +str(path)+'/'+str(item)).size)
        path_img.append(IMG_PATH +str(path)+'/'+str(item))
train_df = pd.DataFrame({'file' : img_file, 'path' : path_img, 'label' : label, 'size' : size})

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_df[['file', 'path', 'size']],
                                                  train_df[['label']],
                                                  train_size = .8,
                                                  stratify = train_df['label'],
                                                  random_state = 2)

train_x2, val_x2, train_y2, val_y2 = train_test_split(train_df[['file', 'path', 'size']],
                                                  train_df[['label']],
                                                  train_size = .8,
                                                  stratify = train_df['label'],
                                                  random_state = 3)

train_x3, val_x3, train_y3, val_y3 = train_test_split(train_df[['file', 'path', 'size']],
                                                  train_df[['label']],
                                                  train_size = .8,
                                                  stratify = train_df['label'],
                                                  random_state = 4)

train_x4, val_x4, train_y4, val_y4 = train_test_split(train_df[['file', 'path', 'size']],
                                                  train_df[['label']],
                                                  train_size = .8,
                                                  stratify = train_df['label'],
                                                  random_state = 5)

train_x5, val_x5, train_y5, val_y5 = train_test_split(train_df[['file', 'path', 'size']],
                                                  train_df[['label']],
                                                  train_size = .8,
                                                  stratify = train_df['label'],
                                                  random_state = 6)
train_set = pd.concat([train_x.reset_index(drop = True),
                       train_y.reset_index(drop = True)],
                      axis = 1)
val_set = pd.concat([val_x.reset_index(drop = True),
                     val_y.reset_index(drop = True)],
                      axis = 1)

"""train_set2 = pd.concat([train_x2.reset_index(drop = True),
                       train_y2.reset_index(drop = True)],
                      axis = 1)
val_set2 = pd.concat([val_x2.reset_index(drop = True),
                     val_y2.reset_index(drop = True)],
                      axis = 1)

train_set3 = pd.concat([train_x3.reset_index(drop = True),
                       train_y3.reset_index(drop = True)],
                      axis = 1)
val_set3 = pd.concat([val_x3.reset_index(drop = True),
                     val_y3.reset_index(drop = True)],
                      axis = 1)

train_set4 = pd.concat([train_x4.reset_index(drop = True),
                       train_y4.reset_index(drop = True)],
                      axis = 1)
val_set4 = pd.concat([val_x4.reset_index(drop = True),
                     val_y4.reset_index(drop = True)],
                      axis = 1)

train_set5 = pd.concat([train_x5.reset_index(drop = True),
                       train_y5.reset_index(drop = True)],
                      axis = 1)
val_set5 = pd.concat([val_x5.reset_index(drop = True),
                     val_y5.reset_index(drop = True)],
                      axis = 1)"""

In [ ]:
!apt-get install tree
## create new folders
!mkdir TRAIN TEST VAL TRAIN/GLIOMA_TUMOR_FIX TRAIN/MENINGIOMA_TUMOR_FIX TRAIN/NO_TUMOR_FIX TRAIN/PITUITARY_TUMOR_FIX VAL/GLIOMA_TUMOR_FIX VAL/MENINGIOMA_TUMOR_FIX VAL/NO_TUMOR_FIX VAL/PITUITARY_TUMOR_FIX
!tree -d


#IMG_PATH = '../input/brain-tumor-classification-mri/Training/'
# split the data by train/val/test
#for CLASS in os.listdir(IMG_PATH):
    #if not CLASS.startswith('.'):
#    IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
#    for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
#        print(FILE_NAME)
        
"""IMG_PATH = '../input/brain-mri-os/Training fix/'
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    #for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
    for FILE_NAME in os.listdir(IMG_PATH+CLASS) :
        img = IMG_PATH + CLASS + '/' + FILE_NAME
        if FILE_NAME in list(train_set['file']) :
            shutil.copy(img, 'TRAIN/' + CLASS.upper() + '/')
        else :
            shutil.copy(img, 'VAL/'+ CLASS.upper() + '/')"""
            

# train no tumor path
for img in train_set[train_set['label'] == 'no_tumor_fix']['path'] :
    shutil.copy(img, 'TRAIN/NO_TUMOR_FIX/')

for img in train_set[train_set['label'] == 'meningioma_tumor_fix']['path'] :
    shutil.copy(img, 'TRAIN/MENINGIOMA_TUMOR_FIX/')
    
for img in train_set[train_set['label'] == 'glioma_tumor_fix']['path'] :
    shutil.copy(img, 'TRAIN/GLIOMA_TUMOR_FIX/')

for img in train_set[train_set['label'] == 'pituitary_tumor_fix']['path'] :
    shutil.copy(img, 'TRAIN/PITUITARY_TUMOR_FIX/')
    
for img in val_set[val_set['label'] == 'no_tumor_fix']['path'] :
    shutil.copy(img, 'VAL/NO_TUMOR_FIX/')

for img in val_set[val_set['label'] == 'meningioma_tumor_fix']['path'] :
    shutil.copy(img, 'VAL/MENINGIOMA_TUMOR_FIX/')
    
for img in val_set[val_set['label'] == 'glioma_tumor_fix']['path'] :
    shutil.copy(img, 'VAL/GLIOMA_TUMOR_FIX/')

for img in val_set[val_set['label'] == 'pituitary_tumor_fix']['path'] :
    shutil.copy(img, 'VAL/PITUITARY_TUMOR_FIX/')

### train val label distribution

In [ ]:
sns.set_style('darkgrid')
fig, ax = plt.subplots(1, 2, figsize = [15, 5])
sns.countplot(train_set[['label']].sort_values(by = 'label', ascending = True)['label'], ax = ax[0])
sns.countplot(val_set[['label']].sort_values(by = 'label', ascending = True)['label'], ax = ax[1])
ax[0].set_xticklabels(['glioma', 'meningioma', 'no_tumor', 'pituitary (hipofisis)'])
ax[1].set_xticklabels(['glioma', 'meningioma', 'no_tumor', 'pituitary (hipofisis)'])
ax[0].set_title('Training Set Label Distribution')
ax[1].set_title('Validation Set Label Distribution')

### showing sample images

In [ ]:
r = 3
c = 3
i = 1
plt.figure(figsize = [15, 15])
plt.suptitle('Pituitary Tumor', y = .92, size = 25)
for path in train_set[train_set['label'] == 'pituitary_tumor_fix'].sample(9, random_state = 2)['path'] :
    plt.subplot(r,c,i)
    img = load_img(path)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    i += 1

In [ ]:
train_notumor_path = glob.glob('./TRAIN/NO_TUMOR_FIX/*.*')
train_meningioma_path = glob.glob('./TRAIN/MENINGIOMA_TUMOR_FIX/*.*')
train_glioma_path = glob.glob('./TRAIN/GLIOMA_TUMOR_FIX/*.*')
train_pituitary_path = glob.glob('./TRAIN/PITUITARY_TUMOR_FIX/*.*')

In [ ]:
val_notumor_path = glob.glob('./VAL/NO_TUMOR_FIX/*.*')
val_meningioma_path = glob.glob('./VAL/MENINGIOMA_TUMOR_FIX/*.*')
val_glioma_path = glob.glob('./VAL/GLIOMA_TUMOR_FIX/*.*')
val_pituitary_path = glob.glob('./VAL/PITUITARY_TUMOR_FIX/*.*')

sorting the path just like train_set and val_set

In [ ]:
train_notumor_path2 = list(train_set[train_set['label'] == 'no_tumor_fix']['path'])
train_meningioma_path2 = list(train_set[train_set['label'] == 'meningioma_tumor_fix']['path'])
train_glioma_path2 = list(train_set[train_set['label'] == 'glioma_tumor_fix']['path'])
train_pituitary_path2 = list(train_set[train_set['label'] == 'pituitary_tumor_fix']['path'])

In [ ]:
val_notumor_path2 = list(val_set[val_set['label'] == 'no_tumor_fix']['path'])
val_meningioma_path2 = list(val_set[val_set['label'] == 'meningioma_tumor_fix']['path'])
val_glioma_path2 = list(val_set[val_set['label'] == 'glioma_tumor_fix']['path'])
val_pituitary_path2 = list(val_set[val_set['label'] == 'pituitary_tumor_fix']['path'])

In [ ]:
#train_glioma
np.random.seed(88)
random_index = list(np.random.randint(1, 100, 5))
random_index

In [ ]:
def grayscale_plot(list_path, title) :
    n_img = len(list_path)
    fig = plt.figure(figsize = [15, (1*n_img)])
    i = 1
    plt.title(title, size = 20)
    plt.xticks([])
    plt.yticks([])
    sns.set_style('dark')
    for path in list_path :
        img = cv2.imread(path)
        ax = fig.add_subplot(2, n_img, i)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY))
        plt.xticks([])
        plt.yticks([])
        #i += 1
        
        ax = fig.add_subplot(2, n_img, i+5)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        i += 1

In [ ]:
grayscale_plot(np.array(train_pituitary_path)[random_index], 'Hipofisis\n')

In [ ]:
#list(train_df['size'].value_counts())
index = [str(i) for i in train_df['size'].value_counts()[:10].index]
index += ['other']
val = list(train_df['size'].value_counts()[:10])
val += [sum(train_df['size'].value_counts()[10:])]
sns.set_style('white')
plt.figure(figsize = [12, 5])
sns.barplot(y = val,
            x = index,
            palette = 'viridis_r')
plt.title('Barplot Banyaknya Gambar dengan Ukuran Tertentu')
for index, value in enumerate(val) :
    plt.text(index-.15, value, value)

### Syntax Resize

In [ ]:
resize_img_path('./VAL', (224, 224))
resize_img_path('./TRAIN', (224, 224))

In [ ]:
tumor_encode_dict = {'no_tumor' : 0,
                     'meningioma' : 1,
                     'glioma' : 2,
                     'pituitary' : 3}

In [ ]:
def crop_plot(nocrop_path, crop_array, title) :
    n_img = len(nocrop_path)
    fig = plt.figure(figsize = [15, (1*n_img)])
    i = 1
    plt.title(title, size = 20)
    plt.xticks([])
    plt.yticks([])
    sns.set_style('dark')
    for nocrop, crop in zip(nocrop_path, crop_array) :
        img = cv2.imread(nocrop)        
        ax = fig.add_subplot(2, n_img, i)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        #i += 1
        
        ax = fig.add_subplot(2, n_img, i+5)
        plt.imshow(crop)
        plt.xticks([])
        plt.yticks([])

        i += 1

In [ ]:
crop_train_notumor = crop_imgs(train_notumor_path)
crop_train_meningioma = crop_imgs(train_meningioma_path)
crop_train_glioma = crop_imgs(train_glioma_path)
crop_train_pituitary = crop_imgs(train_pituitary_path)

In [ ]:
#cv2.imread(train_notumor_path[0], cv2.CV_LOAD_IMAGE_GRAYSCALE)
#cv2.CV_LOAD_IMAGE_GRAYSCALE(train_notumor_path[0])

In [ ]:
#train_notumor[0]
"""X_train = []
y_train = []
for notumor in train_notumor_path :
    image = tf.keras.preprocessing.image.load_img(notumor,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train.append(image)
    y_train.append(0)
    
for meningioma in train_meningioma_path :
    image = tf.keras.preprocessing.image.load_img(meningioma,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train.append(image)
    y_train.append(1)
    
for glioma in train_glioma_path :
    image = tf.keras.preprocessing.image.load_img(glioma,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train.append(image)
    y_train.append(2)
    
for pituitary in train_pituitary_path :
    image = tf.keras.preprocessing.image.load_img(pituitary,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train.append(image)
    y_train.append(3)
    
X_train = np.array(X_train)
y_train = np.array(y_train)"""

In [ ]:
#train_notumor[0]
X_train = []
X_train_name = []
y_train = []
for notumor in train_notumor_path2 :
    dir_ = './TRAIN/NO_TUMOR_FIX/' + notumor.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train_name.append(dir_)
    X_train.append(image)
    y_train.append(0)
    
for meningioma in train_meningioma_path2 :
    dir_ = './TRAIN/MENINGIOMA_TUMOR_FIX/' + meningioma.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train_name.append(dir_)
    X_train.append(image)
    y_train.append(1)
    
for glioma in train_glioma_path2 :
    dir_ = './TRAIN/GLIOMA_TUMOR_FIX/' + glioma.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train_name.append(dir_)
    X_train.append(image)
    y_train.append(2)
    
for pituitary in train_pituitary_path2 :
    dir_ = './TRAIN/PITUITARY_TUMOR_FIX/' + pituitary.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_train_name.append(dir_)
    X_train.append(image)
    y_train.append(3)
    
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
crop_val_notumor = crop_imgs(val_notumor_path)
crop_val_meningioma = crop_imgs(val_meningioma_path)
crop_val_glioma = crop_imgs(val_glioma_path)
crop_val_pituitary = crop_imgs(val_pituitary_path)

In [ ]:
#val_notumor[0]
"""X_val = []
y_val = []
for notumor in val_notumor_path :
    image = tf.keras.preprocessing.image.load_img(notumor,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val.append(image)
    y_val.append(0)
    
for meningioma in val_meningioma_path :
    image = tf.keras.preprocessing.image.load_img(meningioma,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val.append(image)
    y_val.append(1)
    
for glioma in val_glioma_path :
    image = tf.keras.preprocessing.image.load_img(glioma,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val.append(image)
    y_val.append(2)
    
for pituitary in val_pituitary_path :
    image = tf.keras.preprocessing.image.load_img(pituitary,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val.append(image)
    y_val.append(3)
    
X_val = np.array(X_val)
y_val = np.array(y_val)"""

In [ ]:
#val_notumor[0]
X_val = []
X_val_name = []
y_val = []
for notumor in val_notumor_path2 :
    dir_ = './VAL/NO_TUMOR_FIX/' + notumor.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val_name.append(dir_)
    X_val.append(image)
    y_val.append(0)
    
for meningioma in val_meningioma_path2 :
    dir_ = './VAL/MENINGIOMA_TUMOR_FIX/' + meningioma.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val_name.append(dir_)
    X_val.append(image)
    y_val.append(1)
    
for glioma in val_glioma_path2 :
    dir_ = './VAL/GLIOMA_TUMOR_FIX/' + glioma.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val_name.append(dir_)
    X_val.append(image)
    y_val.append(2)
    
for pituitary in val_pituitary_path2 :
    dir_ = './VAL/PITUITARY_TUMOR_FIX/' + pituitary.split('_fix/', 1)[1]
    image = tf.keras.preprocessing.image.load_img(dir_,
                                                  color_mode = 'grayscale')
    image = np.array(image)
    X_val_name.append(dir_)
    X_val.append(image)
    y_val.append(3)
    
X_val = np.array(X_val)
y_val = np.array(y_val)

In [ ]:
var = train_pituitary_path
crop_var = crop_train_pituitary
crop_plot(np.array(var)[random_index], crop_var[random_index], 'Hipofisis')

In [ ]:
#cv2.IMREAD_GRAYSCALE(train_notumor_path[0])


In [ ]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_train,
                                                      y_train,
                                                      train_size = .8,
                                                      random_state = 1)

X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train,
                                                      y_train,
                                                      train_size = .8,
                                                      random_state = 2)

X_train3, X_val3, y_train3, y_val3 = train_test_split(X_train,
                                                      y_train,
                                                      train_size = .8,
                                                      random_state = 3)

X_train4, X_val4, y_train4, y_val4 = train_test_split(X_train,
                                                      y_train,
                                                      train_size = .8,
                                                      random_state = 4)

X_train5, X_val5, y_train5, y_val5 = train_test_split(X_train,
                                                      y_train,
                                                      train_size = .8,
                                                      random_state = 5)

In [ ]:
def inception(x, filters):
    # 1x1
    path1 = Conv2D(filters=filters[0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)

    # 1x1->3x3
    path2 = Conv2D(filters=filters[1][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path2 = Conv2D(filters=filters[1][1], kernel_size=(3,3), strides=1, padding='same', activation='relu')(path2)
    
    # 1x1->5x5
    path3 = Conv2D(filters=filters[2][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path3 = Conv2D(filters=filters[2][1], kernel_size=(5,5), strides=1, padding='same', activation='relu')(path3)

    # 3x3->1x1
    path4 = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(x)
    path4 = Conv2D(filters=filters[3], kernel_size=(1,1), strides=1, padding='same', activation='relu')(path4)

    return Concatenate(axis=-1)([path1,path2,path3,path4])


def auxiliary(x, name=None):
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = Conv2D(filters=128, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu',kernel_regularizer=regularizers.l2(0.0001))(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(units=CLASS_NUM, activation='softmax', name=name)(layer)
    return layer


def googlenet():
    layer_in = Input(shape=IMAGE_SHAPE)
    
    # start of stem mechanism
    # stage-1
    layer = Conv2D(filters=64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(layer_in)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    layer = BatchNormalization()(layer)

    # stage-2
    layer = Conv2D(filters=64, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Conv2D(filters=192, kernel_size=(3,3), strides=1, padding='same', activation='relu')(layer)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    # end of stem mechanism
    
    # stage-3
    layer = inception(layer, [ 64,  (96,128), (16,32), 32]) #3a
    layer = inception(layer, [128, (128,192), (32,96), 64]) #3b
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-4
    layer = inception(layer, [192,  (96,208),  (16,48),  64]) #4a
    aux1  = auxiliary(layer, name='aux1')
    layer = inception(layer, [160, (112,224),  (24,64),  64]) #4b
    layer = inception(layer, [128, (128,256),  (24,64),  64]) #4c
    layer = inception(layer, [112, (144,288),  (32,64),  64]) #4d
    aux2  = auxiliary(layer, name='aux2')
    layer = inception(layer, [256, (160,320), (32,128), 128]) #4e
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-5
    layer = inception(layer, [256, (160,320), (32,128), 128]) #5a
    layer = inception(layer, [384, (192,384), (48,128), 128]) #5b
    layer = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid')(layer)
    
    # stage-6
    layer = Flatten()(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(units=256, activation='linear',kernel_regularizer=regularizers.l2(0.001))(layer)
    main = Dense(units=CLASS_NUM, activation='softmax', name='main')(layer)
    
    model = Model(inputs=layer_in, outputs=[main, aux1, aux2])
    
    return model

In [ ]:
CLASS_NUM = 4
BATCH_SIZE = 8
EPOCH_STEPS = int(X_train.shape[0]/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 1)
MODEL_NAME = 'skripsi_googlenet'

In [ ]:
# train model
model = googlenet()
model.summary()
#model.load_weights(MODEL_NAME)
tf.keras.utils.plot_model(model, 'GoogLeNet.png')

optimizer = Adam(lr = 0.001, 
                 beta_1 = 0.9, 
                 beta_2 = 0.999, 
                 epsilon = 1e-08)
#optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
optimizer = ['Adam', 'SGD', 'Adam', 'SGD']
epochs = [20, 30, 20, 30]
history_all = {}

history = model.fit(X_train,
                    y_train,
                    epochs = 50,
                    steps_per_epoch = EPOCH_STEPS,
                    validation_data = (X_val,y_val))

model.save(MODEL_NAME)

In [ ]:
plt.figure(figsize = [15, 5])
sns.set_style('whitegrid')
plt.plot(history.history['main_accuracy'])
plt.plot(history.history['val_main_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize = [15, 5])
plt.plot(history.history['main_loss'])
plt.plot(history.history['val_main_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(X_val, y_val)
print('Score:', score[4])#akurasi

print('\n loss')
print('Score:', score[1])

In [ ]:
res = prob_converter(model.predict(X_val))
res_average = prob_average(model.predict(X_val))
print(confusion_matrix(np.array(res), np.array(y_val)))
print(accuracy_score(np.array(res), np.array(y_val)))

In [ ]:
# train model
CLASS_NUM = 4
BATCH_SIZE = 8
EPOCH_STEPS = int(X_train1.shape[0]/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 1)
MODEL_NAME = 'skripsi_googlenet'


model = googlenet()
model.summary()
#model.load_weights(MODEL_NAME)
tf.keras.utils.plot_model(model, 'GoogLeNet.png')

optimizer = Adam(lr = 0.001, 
                 beta_1 = 0.9, 
                 beta_2 = 0.999, 
                 epsilon = 1e-08)
#optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
optimizer = ['Adam', 'SGD', 'Adam', 'SGD']
epochs = [20, 30, 20, 30]
history_all = {}

history = model.fit(X_train1,
                    y_train1,
                    epochs = 50,
                    steps_per_epoch = EPOCH_STEPS,
                    validation_data = (X_val1,y_val1))

model.save(MODEL_NAME)

res = prob_converter(model.predict(X_val1))
res_average = prob_average(model.predict(X_val1))
print(confusion_matrix(np.array(res), np.array(y_val1)))
print(accuracy_score(np.array(res), np.array(y_val1)))

In [ ]:
# train model
CLASS_NUM = 4
BATCH_SIZE = 8
EPOCH_STEPS = int(X_train2.shape[0]/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 1)
MODEL_NAME = 'skripsi_googlenet'


model = googlenet()
model.summary()
#model.load_weights(MODEL_NAME)
tf.keras.utils.plot_model(model, 'GoogLeNet.png')

optimizer = Adam(lr = 0.001, 
                 beta_1 = 0.9, 
                 beta_2 = 0.999, 
                 epsilon = 1e-08)
#optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
optimizer = ['Adam', 'SGD', 'Adam', 'SGD']
epochs = [20, 30, 20, 30]
history_all = {}

history = model.fit(X_train2,
                    y_train2,
                    epochs = 50,
                    steps_per_epoch = EPOCH_STEPS,
                    validation_data = (X_val2,y_val2))

model.save(MODEL_NAME)

res2 = prob_converter(model.predict(X_val2))
res_average2 = prob_average(model.predict(X_val2))
print(confusion_matrix(np.array(res2), np.array(y_val2)))
print(accuracy_score(np.array(res2), np.array(y_val2)))

In [ ]:
# train model
CLASS_NUM = 4
BATCH_SIZE = 8
EPOCH_STEPS = int(X_train3.shape[0]/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 1)
MODEL_NAME = 'skripsi_googlenet'



model = googlenet()
model.summary()
#model.load_weights(MODEL_NAME)
tf.keras.utils.plot_model(model, 'GoogLeNet.png')

optimizer = Adam(lr = 0.001, 
                 beta_1 = 0.9, 
                 beta_2 = 0.999, 
                 epsilon = 1e-08)
#optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
optimizer = ['Adam', 'SGD', 'Adam', 'SGD']
epochs = [20, 30, 20, 30]
history_all = {}

history = model.fit(X_train3,
                    y_train3,
                    epochs = 50,
                    steps_per_epoch = EPOCH_STEPS,
                    validation_data = (X_val3,y_val3))

model.save(MODEL_NAME)

res3 = prob_converter(model.predict(X_val3))
res_average3 = prob_average(model.predict(X_val3))
print(confusion_matrix(np.array(res3), np.array(y_val3)))
print(accuracy_score(np.array(res3), np.array(y_val3)))

In [ ]:
# train model
CLASS_NUM = 4
BATCH_SIZE = 8
EPOCH_STEPS = int(X_train4.shape[0]/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 1)
MODEL_NAME = 'skripsi_googlenet'


model = googlenet()
model.summary()
#model.load_weights(MODEL_NAME)
tf.keras.utils.plot_model(model, 'GoogLeNet.png')

optimizer = Adam(lr = 0.001, 
                 beta_1 = 0.9, 
                 beta_2 = 0.999, 
                 epsilon = 1e-08)
#optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
optimizer = ['Adam', 'SGD', 'Adam', 'SGD']
epochs = [20, 30, 20, 30]
history_all = {}

history = model.fit(X_train4,
                    y_train4,
                    epochs = 50,
                    steps_per_epoch = EPOCH_STEPS,
                    validation_data = (X_val4,y_val4))

model.save(MODEL_NAME)

res4 = prob_converter(model.predict(X_val4))
res_average4 = prob_average(model.predict(X_val4))
print(confusion_matrix(np.array(res4), np.array(y_val4)))
print(accuracy_score(np.array(res4), np.array(y_val4)))

In [ ]:
# train model
CLASS_NUM = 4
BATCH_SIZE = 8
EPOCH_STEPS = int(X_train5.shape[0]/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 1)
MODEL_NAME = 'skripsi_googlenet'



model = googlenet()
model.summary()
#model.load_weights(MODEL_NAME)
tf.keras.utils.plot_model(model, 'GoogLeNet.png')

optimizer = Adam(lr = 0.001, 
                 beta_1 = 0.9, 
                 beta_2 = 0.999, 
                 epsilon = 1e-08)
#optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
optimizer = ['Adam', 'SGD', 'Adam', 'SGD']
epochs = [20, 30, 20, 30]
history_all = {}

history = model.fit(X_train5,
                    y_train5,
                    epochs = 50,
                    steps_per_epoch = EPOCH_STEPS,
                    validation_data = (X_val5,y_val5))

model.save(MODEL_NAME)

res5 = prob_converter(model.predict(X_val5))
res_average5 = prob_average(model.predict(X_val5))
print(confusion_matrix(np.array(res5), np.array(y_val5)))
print(accuracy_score(np.array(res5), np.array(y_val5)))

In [ ]:
#df_result = pd.DataFrame({'actual' : y_val,
                          #'pred' : res,
                          #'prob' : [list(i) for i in res_average]})
#df_result.to_csv('pred result 9 jan 2.csv', index = False)

In [ ]:
def sample_plot(nrow, ncol, data, index) :
    fig = plt.figure(figsize = [3*ncol, 5*nrow])
    i = 1
    for index_ in index :
        ax = fig.add_subplot(nrow, ncol, i)
        plt.imshow(data[index_],
                   cmap = 'gray')
        plt.xticks([])
        plt.yticks([])
        plt.text(i-1, -2, index_, size = 30)
        i += 1

In [ ]:
def image_to_vector_converter(data) : 
    dim = data[0].shape[0]
    result = []
    for img in tqdm(data) :
        res = []
        for i in list(range(dim)) :
            res += list(img[i])
        result.append(res)
    return pd.DataFrame(np.array(result))

In [ ]:
X_train_df = image_to_vector_converter(X_train)
X_val_df = image_to_vector_converter(X_val)

In [ ]:
X_train_df1 = image_to_vector_converter(X_train1)
X_val_df1 = image_to_vector_converter(X_val1)

X_train_df2 = image_to_vector_converter(X_train2)
X_val_df2 = image_to_vector_converter(X_val2)

X_train_df3 = image_to_vector_converter(X_train3)
X_val_df3 = image_to_vector_converter(X_val3)

X_train_df4 = image_to_vector_converter(X_train4)
X_val_df4 = image_to_vector_converter(X_val4)

X_train_df5 = image_to_vector_converter(X_train5)
X_val_df5 = image_to_vector_converter(X_val5)

In [ ]:
classifiers = [LogisticRegression(),
               GaussianNB(),
               SVC(),
               RandomForestClassifier(random_state = 2),
               DecisionTreeClassifier(random_state = 2)]

acc_score = []
var_score = []
algos = []
for classifier in tqdm(classifiers) :
    algos.append(type(classifier).__name__)
    result = []
    
    # set 1
    classifier.fit(X_train_df1,
                   y_train1)
    pred = classifier.predict(X_val_df1)
    result.append(accuracy_score(np.array(pred), np.array(y_val1)))
    
    # set 2
    classifier.fit(X_train_df2,
                   y_train2)
    pred = classifier.predict(X_val_df2)
    result.append(accuracy_score(np.array(pred), np.array(y_val2)))
    
    # set 3
    classifier.fit(X_train_df3,
                   y_train3)
    pred = classifier.predict(X_val_df3)
    result.append(accuracy_score(np.array(pred), np.array(y_val3)))
    
    # set 4
    classifier.fit(X_train_df4,
                   y_train4)
    pred = classifier.predict(X_val_df4)
    result.append(accuracy_score(np.array(pred), np.array(y_val4)))
    
    # set 2
    classifier.fit(X_train_df5,
                   y_train5)
    pred = classifier.predict(X_val_df5)
    result.append(accuracy_score(np.array(pred), np.array(y_val5)))
    
    result = np.array(result)
    
    acc_score.append(result.mean())
    var_score.append(result.var())

In [ ]:
pd.DataFrame({'method' : algos,
              'accuracy' : acc_score,
              'variance' : var_score})

algos.append('CNN & GoogLeNet')
acc_score.append(np.array([accuracy_score(np.array(res), np.array(y_val1)),
          accuracy_score(np.array(res2), np.array(y_val2)),
          accuracy_score(np.array(res3), np.array(y_val3)),
          accuracy_score(np.array(res4), np.array(y_val4)),
          accuracy_score(np.array(res5), np.array(y_val5))]).mean())

var_score.append(np.array([accuracy_score(np.array(res), np.array(y_val1)),
          accuracy_score(np.array(res2), np.array(y_val2)),
          accuracy_score(np.array(res3), np.array(y_val3)),
          accuracy_score(np.array(res4), np.array(y_val4)),
          accuracy_score(np.array(res5), np.array(y_val5))]).var())

classifiers_df_cv = pd.DataFrame({'method' : algos,
                                  'accuracy' : acc_score,
                                  'variance' : var_score})

In [ ]:
plt.figure(figsize = [15, 8])
sns.barplot(data = classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False),
            x = 'accuracy',
            y = 'method',
            palette = 'viridis')

for index, value in enumerate(classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['accuracy']) :
    plt.text(value, index, '%.3f'%value)
    
for index, value in enumerate(classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance']) :
    plt.text(0, index, 'var : ' + '%.4f'%value)
plt.title('Barplot of Cross Validation Performance', size = 20)

In [ ]:
classifiers = [LogisticRegression(),
               GaussianNB(),
               SVC(),
               RandomForestClassifier(random_state = 2),
               DecisionTreeClassifier(random_state = 2)]
acc_score = []
conf_list = []
algos = []
for classifier in tqdm(classifiers) :
    model = classifier.fit(X_train_df,
                           y_train)
    pred = classifier.predict(X_val_df)
    algos.append(type(classifier).__name__)
    acc_score.append(accuracy_score(np.array(pred), np.array(y_val)))
    conf_list.append(list(confusion_matrix(np.array(pred), np.array(y_val))))
classifiers_df = pd.DataFrame({'method' : algos,
                               'accuracy' : acc_score,
                               'conf_list' : conf_list})

In [ ]:
classifiers_df

In [ ]:
classifiers = [LogisticRegression(),
               GaussianNB(),
               SVC(),
               RandomForestClassifier(random_state = 2),
               DecisionTreeClassifier(random_state = 2)]

cv_accuracy = []
cv_var = []
for classifier in tqdm(classifiers) :
    model = classifier.fit(X_train_df,
                           y_train)
    #pred = classifier.predict(X_val_df)
    #algos.append(type(classifier).__name__)
    #acc_score.append(accuracy_score(np.array(pred), np.array(y_val)))
    #conf_list.append(list(confusion_matrix(np.array(pred), np.array(y_val))))
    cv = cross_val_score(model,
                         X_train_df,
                         y_train,
                         scoring = 'accuracy',
                         cv = KFold(5))
    cv_accuracy.append(cv.mean())
    cv_var.append(cv.var())
classifiers_df_cv = pd.DataFrame({'method' : algos,
                                  'accuracy' : cv_accuracy,
                                  'conf_list' : cv_var})

In [ ]:
#classifiers_df_cv
algos.append('CNN & GoogLeNet')
cv_accuracy.append(np.array([accuracy_score(np.array(res), np.array(y_val1)),
          accuracy_score(np.array(res2), np.array(y_val2)),
          accuracy_score(np.array(res3), np.array(y_val3)),
          accuracy_score(np.array(res4), np.array(y_val4)),
          accuracy_score(np.array(res5), np.array(y_val5))]).mean())

cv_var.append(np.array([accuracy_score(np.array(res), np.array(y_val1)),
          accuracy_score(np.array(res2), np.array(y_val2)),
          accuracy_score(np.array(res3), np.array(y_val3)),
          accuracy_score(np.array(res4), np.array(y_val4)),
          accuracy_score(np.array(res5), np.array(y_val5))]).var())

classifiers_df_cv = pd.DataFrame({'method' : algos,
                                  'accuracy' : cv_accuracy,
                                  'variance' : cv_var})

In [ ]:
%matplotlib inline
sns.set()
plt.figure(figsize = [15, 8])
sns.set_style('whitegrid')
sns.barplot(data = classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False),
            x = 'accuracy',
            y = 'method',
            palette = 'viridis',
            ci = 'sd')

#labels = ['%.4f'%classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance'][5],
                     #'%.4f'%classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance'][4],
                     #'%.4f'%classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance'][0],
                     #'%.4f'%classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance'][2],
                     #'%.4f'%classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance'][1]]
#classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance'][0]


plt.title('Barplot of Cross Validation Result', size = 20)
#bars = [r for r in ax.get_children() if type(r)==Rectangle]
#colors = [c.get_facecolor() for c in bars[:-1]] # I think the last Rectangle is the background
for index, value in enumerate(classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['accuracy']) :
    plt.text(value, index, '%.2f'%value)

for index, value in enumerate(classifiers_df_cv.drop(3).sort_values(by = 'accuracy', ascending = False)['variance']) :
    plt.text(0, index, 'var : ' + '%.4f'%value)

In [ ]:
classifiers = [LogisticRegression(),
               GaussianNB(),
               SVC(),
               RandomForestClassifier(random_state = 2),
               DecisionTreeClassifier(random_state = 2)]
algos = []
accuracy = []
for classifier in tqdm(classifiers) :
    algos.append(type(classifier).__name__)
    classifier.fit(X_train_df,
                   y_train)
    pred = classifier.predict(X_val_df)
    accuracy.append(accuracy_score(np.array(pred), y_val))
vis_df = pd.DataFrame({'method' : algos,
                       'accuracy' : accuracy})
vis_df

In [ ]:
#vis_df.append(pd.DataFrame(['CNN & GoogLeNet', 0.953917]), columns = ['method', 'accuracy'])
vis_df.append(pd.DataFrame({'method' : 'CNN & GoogLeNet',
                            'accuracy' : [0.953917]}))

In [ ]:
plt.figure(figsize = [15, 8])
sns.barplot(data = vis_df.append(pd.DataFrame({'method' : 'CNN & GoogLeNet',
                            'accuracy' : [0.953917]})).drop(3).sort_values(by = 'accuracy',
                                                                           ascending = False),
            x = 'accuracy',
            y = 'method',
            palette = 'viridis')
for index, value in enumerate(vis_df.append(pd.DataFrame({'method' : 'CNN & GoogLeNet',
                            'accuracy' : [0.953917]})).drop(3).sort_values(by = 'accuracy',
                                                                           ascending = False)['accuracy']) :
    plt.text(value, index, '%.3f'%value)
plt.title('Classifiers Comparison on Validation Set', size = 20)